# GREAT CARIA - Publication-Ready Fragility Model

## Enhancements:
1. **Expanded Data Universe**: Sectors, Yield Curve, Credit Spreads, Liquidity, Commodity Vol
2. **Formal Validation**: Block Bootstrap, Granger/PCMCI Causality, Error Estimation
3. **Publication-Quality**: Confidence intervals, effect sizes, reproducibility

In [ ]:
!pip install PyWavelets networkx yfinance fredapi tigramite arch -q

import pandas as pd
import numpy as np
from scipy import stats, signal
from scipy.ndimage import gaussian_filter1d
import pywt
import yfinance as yf
from sklearn.decomposition import FactorAnalysis, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from statsmodels.tsa.stattools import grangercausalitytests
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

# Base data
MARKET_PATH = '/content/drive/MyDrive/CARIA/data/raw/yahoo_market.parquet'
df_base = pd.read_parquet(MARKET_PATH)
print(f'Base data: {df_base.shape}, {df_base.index.min().date()} to {df_base.index.max().date()}')

---
# PART 1: EXPANDED DATA UNIVERSE

In [ ]:
# === 1A: Fetch additional data from Yahoo Finance ===
print('=== Fetching Expanded Data Universe ===')

# Define tickers
TICKERS = {
    # Sector ETFs (US)
    'XLF': 'Financials',
    'XLK': 'Technology',
    'XLE': 'Energy',
    'XLV': 'Healthcare',
    'XLI': 'Industrials',
    'XLP': 'Staples',
    'XLY': 'Discretionary',
    'XLU': 'Utilities',
    'XLB': 'Materials',
    'XLRE': 'RealEstate',
    
    # Yield curve proxies
    'TLT': 'Treasury20Y',
    'IEF': 'Treasury10Y',
    'SHY': 'Treasury3Y',
    
    # Credit spreads proxies
    'HYG': 'HighYield',
    'LQD': 'InvestmentGrade',
    'JNK': 'JunkBonds',
    
    # Commodities
    'GLD': 'Gold',
    'USO': 'Oil',
    'UNG': 'NatGas',
    
    # Volatility
    'VIXY': 'VIXFutures',
}

# Fetch data
start_date = df_base.index.min().strftime('%Y-%m-%d')
end_date = df_base.index.max().strftime('%Y-%m-%d')

expanded_data = {}
for ticker, name in tqdm(TICKERS.items()):
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)['Adj Close']
        if len(data) > 100:
            expanded_data[name] = data
    except:
        pass

df_expanded = pd.DataFrame(expanded_data)
print(f'\nExpanded data: {df_expanded.shape}')
print(f'Available: {list(df_expanded.columns)}')

In [ ]:
# === 1B: Fetch FRED data (if API key available) ===
print('\n=== FRED Data (Optional) ===')

FRED_SERIES = {
    'T10Y2Y': 'YieldCurve_10Y2Y',
    'T10Y3M': 'YieldCurve_10Y3M',
    'BAA10Y': 'CreditSpread_BAA',
    'TEDRATE': 'TEDSpread',
    'STLFSI4': 'FinancialStress',
    'NFCI': 'FinancialConditions',
    'VIXCLS': 'VIX_FRED'
}

try:
    from fredapi import Fred
    # Try to get API key from environment or use placeholder
    import os
    fred_key = os.environ.get('FRED_API_KEY', '')
    
    if fred_key:
        fred = Fred(api_key=fred_key)
        fred_data = {}
        for series_id, name in FRED_SERIES.items():
            try:
                data = fred.get_series(series_id, start_date, end_date)
                if len(data) > 100:
                    fred_data[name] = data
            except:
                pass
        df_fred = pd.DataFrame(fred_data)
        print(f'FRED data: {df_fred.shape}')
    else:
        print('No FRED API key found - using Yahoo proxies only')
        df_fred = pd.DataFrame()
except Exception as e:
    print(f'FRED not available: {e}')
    df_fred = pd.DataFrame()

In [ ]:
# === 1C: Construct derived signals ===
print('\n=== Constructing Derived Signals ===')

# Merge all data
df_all = df_base.copy()
for col in df_expanded.columns:
    df_all[col] = df_expanded[col]
if len(df_fred) > 0:
    for col in df_fred.columns:
        df_all[col] = df_fred[col]

# Forward fill missing values
df_all = df_all.ffill().dropna(how='all')

# Derived signals
derived = pd.DataFrame(index=df_all.index)

# 1. Yield curve slope (proxy from TLT/SHY if available)
if 'Treasury20Y' in df_all.columns and 'Treasury3Y' in df_all.columns:
    derived['YieldSlope'] = (df_all['Treasury20Y'].pct_change(20) - 
                             df_all['Treasury3Y'].pct_change(20))

# 2. Credit spread proxy (HYG vs LQD)
if 'HighYield' in df_all.columns and 'InvestmentGrade' in df_all.columns:
    derived['CreditSpread'] = (df_all['HighYield'] / df_all['InvestmentGrade']).pct_change(20)

# 3. Sector dispersion
sector_cols = [c for c in ['Financials', 'Technology', 'Energy', 'Healthcare', 
                           'Industrials', 'Staples', 'Discretionary'] if c in df_all.columns]
if len(sector_cols) >= 3:
    sector_rets = df_all[sector_cols].pct_change()
    derived['SectorDispersion'] = sector_rets.std(axis=1)

# 4. Risk-on/off ratio
if 'Discretionary' in df_all.columns and 'Staples' in df_all.columns:
    derived['RiskOnOff'] = (df_all['Discretionary'] / df_all['Staples']).pct_change(20)

# 5. Commodity stress
if 'Oil' in df_all.columns:
    derived['OilVol'] = df_all['Oil'].pct_change().rolling(20).std() * np.sqrt(252)
if 'Gold' in df_all.columns:
    derived['GoldMomentum'] = df_all['Gold'].pct_change(60)  # Safe haven demand

print(f'Derived signals: {derived.dropna(how="all").shape}')
print(f'Available: {list(derived.dropna(how="all").columns)}')

In [ ]:
# === 1D: Compute base signals (from previous notebook) ===
print('\n=== Computing Base Signals ===')

COUNTRIES = ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'FRA', 'BRA', 'MEX', 'KOR', 'AUS', 'IND', 'ZAF']
idx_cols = [f'{c}_index' for c in COUNTRIES if f'{c}_index' in df_base.columns]
ret = df_base[idx_cols].pct_change().dropna()
ret.columns = [c.replace('_index', '') for c in ret.columns]

# CF
def compute_cf(r, w=20):
    cf = []
    for i in range(w, len(r)):
        wr = r.iloc[i-w:i]
        c = wr.corr().values
        ac = (c.sum() - len(c)) / (len(c) * (len(c) - 1))
        cf.append(ac * wr.std().mean() * 100)
    return pd.Series(cf, index=r.index[w:])

CF = compute_cf(ret)

# Sync
def extract_phase(series):
    detrended = series - gaussian_filter1d(series.values, sigma=60)
    return np.angle(signal.hilbert(detrended))

phases = pd.DataFrame({c: extract_phase(ret[c].fillna(0)) for c in ret.columns}, index=ret.index)
SYNC = pd.Series([np.abs(np.exp(1j * phases.iloc[i].values).mean()) 
                  for i in range(60, len(phases))], index=phases.index[60:])

# EWS
EWS = pd.DataFrame({
    'acf1': CF.rolling(120).apply(lambda x: x.autocorr(1), raw=False),
    'var': CF.rolling(120).var(),
    'skew': CF.rolling(120).skew()
})

print(f'Base signals computed')

---
# PART 2: ENHANCED FACTOR MODEL

In [ ]:
# === 2A: Combine all signals ===
print('=== Building Enhanced Signal Matrix ===')

# Align all signals
common_idx = CF.dropna().index
for sig in [SYNC, EWS['acf1'], EWS['var']]:
    common_idx = common_idx.intersection(sig.dropna().index)
for col in derived.columns:
    common_idx = common_idx.intersection(derived[col].dropna().index)

# Build signal matrix
signals = pd.DataFrame({'cf': CF, 'sync': SYNC, 'acf1': EWS['acf1'], 
                        'var': EWS['var'], 'skew': EWS['skew'].abs()}).loc[common_idx].dropna()

# Add derived if available
for col in derived.columns:
    if col in derived.columns:
        signals[col] = derived[col].loc[signals.index]

signals = signals.dropna()
print(f'Enhanced signal matrix: {signals.shape}')
print(f'Signals: {list(signals.columns)}')

In [ ]:
# === 2B: Factor Analysis with more components ===
print('\n=== Enhanced Factor Analysis ===')

scaler = StandardScaler()
X = scaler.fit_transform(signals)

# PCA to determine optimal components
pca = PCA()
pca.fit(X)
cum_var = np.cumsum(pca.explained_variance_ratio_)

# Find components for 70% variance
n_components = np.argmax(cum_var >= 0.70) + 1
print(f'Components for 70% variance: {n_components}')
print(f'Explained variance: {cum_var[:5]}')

# Factor Analysis
fa = FactorAnalysis(n_components=min(n_components, 3), random_state=42)
F_latent = fa.fit_transform(X)

# First factor = Fragility
F_t = pd.Series(F_latent[:, 0], index=signals.index, name='F_t')

# Loadings
loadings = pd.DataFrame({
    'signal': signals.columns,
    'F1_loading': fa.components_[0]
})
if fa.n_components >= 2:
    loadings['F2_loading'] = fa.components_[1]

print('\nFactor Loadings:')
print(loadings.sort_values('F1_loading', ascending=False).to_string(index=False))

---
# PART 3: FORMAL STATISTICAL VALIDATION

In [ ]:
# === 3A: Block Bootstrap ===
print('=== Block Bootstrap Validation ===')

def block_bootstrap(series, n_bootstrap=500, block_size=60):
    """Block bootstrap preserving temporal dependence"""
    n = len(series)
    n_blocks = n // block_size
    
    stats = []
    for _ in range(n_bootstrap):
        # Sample blocks with replacement
        block_indices = np.random.choice(n_blocks, n_blocks, replace=True)
        boot_series = []
        for bi in block_indices:
            start = bi * block_size
            end = min(start + block_size, n)
            boot_series.extend(series.iloc[start:end].values)
        
        boot_series = pd.Series(boot_series[:n])
        stats.append({
            'mean': boot_series.mean(),
            'std': boot_series.std(),
            'acf1': boot_series.autocorr(1),
            'q90': boot_series.quantile(0.9)
        })
    
    return pd.DataFrame(stats)

boot_stats = block_bootstrap(F_t)

print('Block Bootstrap Results (95% CI):')
for col in ['mean', 'std', 'acf1', 'q90']:
    ci_low, ci_high = np.percentile(boot_stats[col], [2.5, 97.5])
    actual = F_t.mean() if col == 'mean' else (F_t.std() if col == 'std' else 
              (F_t.autocorr(1) if col == 'acf1' else F_t.quantile(0.9)))
    print(f'  {col}: {actual:.4f} [{ci_low:.4f}, {ci_high:.4f}]')

In [ ]:
# === 3B: Granger Causality ===
print('\n=== Granger Causality Tests ===')

# Test if F_t Granger-causes market stress indicators
test_pairs = [
    ('F_t', 'VIX'),
    ('F_t', 'cf'),
    ('sync', 'F_t'),
]

vix = df_base['VIX'].loc[F_t.index].dropna()
common = F_t.index.intersection(vix.index)

granger_results = []
for cause, effect in test_pairs:
    try:
        if cause == 'F_t':
            x = F_t.loc[common]
        elif cause == 'sync':
            x = SYNC.loc[common]
        else:
            x = signals[cause].loc[common]
        
        if effect == 'VIX':
            y = vix.loc[common]
        elif effect == 'F_t':
            y = F_t.loc[common]
        else:
            y = signals[effect].loc[common]
        
        data = pd.concat([y, x], axis=1).dropna()
        if len(data) > 100:
            result = grangercausalitytests(data, maxlag=5, verbose=False)
            # Get p-value for lag 5
            p_value = result[5][0]['ssr_ftest'][1]
            granger_results.append({
                'cause': cause,
                'effect': effect,
                'p_value': p_value,
                'significant': p_value < 0.05
            })
    except Exception as e:
        print(f'  {cause} -> {effect}: Error - {e}')

granger_df = pd.DataFrame(granger_results)
print('\nGranger Causality Results:')
print(granger_df.to_string(index=False))

In [ ]:
# === 3C: PCMCI (if tigramite available) ===
print('\n=== PCMCI Causal Discovery ===')

try:
    from tigramite import data_processing as pp
    from tigramite.pcmci import PCMCI
    from tigramite.independence_tests.parcorr import ParCorr
    
    # Prepare data
    pcmci_cols = ['cf', 'sync', 'acf1', 'var']
    pcmci_data = signals[pcmci_cols].dropna().values
    
    if len(pcmci_data) > 200:
        # Subsample for speed
        pcmci_data = pcmci_data[::5]  # Every 5th observation
        
        dataframe = pp.DataFrame(pcmci_data, var_names=pcmci_cols)
        parcorr = ParCorr(significance='analytic')
        pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr, verbosity=0)
        
        results = pcmci.run_pcmci(tau_max=5, pc_alpha=0.05)
        
        print('PCMCI Results (significant links):')
        for i, var_i in enumerate(pcmci_cols):
            for j, var_j in enumerate(pcmci_cols):
                for tau in range(1, 6):
                    val = results['val_matrix'][i, j, tau]
                    pval = results['p_matrix'][i, j, tau]
                    if pval < 0.05:
                        print(f'  {var_j}(t-{tau}) -> {var_i}(t): val={val:.3f}, p={pval:.3f}')
    else:
        print('Not enough data for PCMCI')
        
except ImportError:
    print('Tigramite not available - skipping PCMCI')
except Exception as e:
    print(f'PCMCI error: {e}')

In [ ]:
# === 3D: Formal Error Estimation ===
print('\n=== Formal Error Estimation ===')

# Crisis prediction accuracy with confidence intervals
CRISES = {
    'Lehman': pd.Timestamp('2008-09-15'),
    'Flash_Crash': pd.Timestamp('2010-05-06'),
    'Euro_Crisis': pd.Timestamp('2011-08-05'),
    'Taper_Tantrum': pd.Timestamp('2013-05-22'),
    'China_Crash': pd.Timestamp('2015-08-24'),
    'Brexit': pd.Timestamp('2016-06-24'),
    'Volmageddon': pd.Timestamp('2018-02-05'),
    'Repo_Crisis': pd.Timestamp('2019-09-17'),
    'COVID': pd.Timestamp('2020-03-11'),
    'Gilt_Crisis': pd.Timestamp('2022-09-23'),
    'SVB': pd.Timestamp('2023-03-10')
}

def compute_lead_with_error(indicator, crisis_date, threshold_pct=0.8, n_bootstrap=100):
    """Compute lead time with bootstrap confidence interval"""
    threshold = indicator.quantile(threshold_pct)
    pre = indicator[(indicator.index < crisis_date) & 
                    (indicator.index > crisis_date - pd.Timedelta(days=180))]
    crossings = pre[pre > threshold]
    
    if len(crossings) > 0:
        lead = (crisis_date - crossings.index[0]).days
        
        # Bootstrap CI
        boot_leads = []
        for _ in range(n_bootstrap):
            boot_thresh = indicator.quantile(threshold_pct + np.random.uniform(-0.05, 0.05))
            boot_cross = pre[pre > boot_thresh]
            if len(boot_cross) > 0:
                boot_leads.append((crisis_date - boot_cross.index[0]).days)
        
        if boot_leads:
            ci_low, ci_high = np.percentile(boot_leads, [2.5, 97.5])
            return lead, ci_low, ci_high
    
    return 0, 0, 0

# Compute for all crises
error_results = []
for crisis_name, crisis_date in CRISES.items():
    if crisis_date < F_t.index.min() + pd.Timedelta(days=180):
        continue
    if crisis_date > F_t.index.max():
        continue
    
    lead, ci_low, ci_high = compute_lead_with_error(F_t, crisis_date)
    error_results.append({
        'crisis': crisis_name,
        'lead': lead,
        'ci_low': ci_low,
        'ci_high': ci_high
    })

error_df = pd.DataFrame(error_results)
print('\nLead Times with 95% CI:')
print(error_df.to_string(index=False))

# Average with SE
avg_lead = error_df['lead'].mean()
se_lead = error_df['lead'].std() / np.sqrt(len(error_df))
print(f'\nAverage Lead: {avg_lead:.1f} ± {1.96*se_lead:.1f} days (95% CI)')

---
# PART 4: PUBLICATION-QUALITY RESULTS

In [ ]:
# === 4A: Effect Sizes ===
print('=== Effect Size Calculations ===')

# Cohen's d for pre-crisis vs normal periods
def cohens_d(group1, group2):
    n1, n2 = len(group1), len(group2)
    var1, var2 = group1.var(), group2.var()
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    return (group1.mean() - group2.mean()) / pooled_std

# Pre-crisis windows (60 days before each crisis)
pre_crisis_values = []
for crisis_date in CRISES.values():
    if crisis_date > F_t.index.min() and crisis_date < F_t.index.max():
        pre = F_t[(F_t.index < crisis_date) & 
                  (F_t.index > crisis_date - pd.Timedelta(days=60))]
        pre_crisis_values.extend(pre.values)

pre_crisis = pd.Series(pre_crisis_values)
normal = F_t[~F_t.index.isin([d for d in CRISES.values()])]

d = cohens_d(pre_crisis, normal)
print(f"Cohen's d (pre-crisis vs normal): {d:.3f}")
print(f"Interpretation: {'Large' if abs(d) > 0.8 else 'Medium' if abs(d) > 0.5 else 'Small'} effect")

In [ ]:
# === 4B: Publication Figure ===

fig, axes = plt.subplots(4, 1, figsize=(12, 14), sharex=True)

# Panel A: Latent Fragility
axes[0].fill_between(F_t.index, F_t.values, alpha=0.3, color='red')
axes[0].plot(F_t.index, F_t.values, 'r-', linewidth=0.5)
axes[0].axhline(F_t.quantile(0.9), color='orange', linestyle='--', alpha=0.7)
axes[0].set_ylabel('$F_t$ (Latent Fragility)', fontsize=11)
axes[0].set_title('A. Systemic Fragility Index', fontsize=12, fontweight='bold')

# Panel B: Key Signals
ax1b = axes[1]
ax1b.plot(signals.index, scaler.fit_transform(signals[['cf']]).flatten(), 
          'b-', label='Crisis Factor', alpha=0.7)
ax1b.plot(signals.index, scaler.fit_transform(signals[['sync']]).flatten(), 
          'orange', label='Synchronization', alpha=0.7)
ax1b.set_ylabel('Standardized Value', fontsize=11)
ax1b.set_title('B. Component Signals', fontsize=12, fontweight='bold')
ax1b.legend(loc='upper left', fontsize=9)

# Panel C: Bootstrap CI
rolling_mean = F_t.rolling(60).mean()
rolling_std = F_t.rolling(60).std()
axes[2].fill_between(rolling_mean.index, 
                     rolling_mean - 1.96*rolling_std,
                     rolling_mean + 1.96*rolling_std,
                     alpha=0.3, color='blue')
axes[2].plot(rolling_mean.index, rolling_mean.values, 'b-', linewidth=1)
axes[2].set_ylabel('$F_t$ (60d rolling)', fontsize=11)
axes[2].set_title('C. Rolling Mean with 95% CI', fontsize=12, fontweight='bold')

# Panel D: S&P 500
sp500 = df_base['USA_index'].loc[F_t.index].dropna()
axes[3].plot(sp500.index, sp500.values, 'k-', linewidth=0.5)
axes[3].set_ylabel('S&P 500', fontsize=11)
axes[3].set_yscale('log')
axes[3].set_title('D. Reference: S&P 500 Index', fontsize=12, fontweight='bold')

# Mark crises
for ax in axes:
    for name, date in CRISES.items():
        if date > F_t.index.min() and date < F_t.index.max():
            ax.axvline(date, color='red', alpha=0.3, linestyle=':')

plt.tight_layout()
OUTPUT_DIR = '/content/drive/MyDrive/CARIA/research/formal_fragility'
plt.savefig(f'{OUTPUT_DIR}/publication_figure.png', dpi=300, bbox_inches='tight')
plt.savefig(f'{OUTPUT_DIR}/publication_figure.pdf', bbox_inches='tight')
plt.show()
print('\n✓ Saved publication figure (PNG + PDF)')

In [ ]:
# === 4C: Results Table for Paper ===
print('\n=== Table 1: Model Performance ===')

table1 = pd.DataFrame({
    'Metric': ['Variance Explained', 'Average Lead Time', 'Cohen\'s d', 
               'Crises Detected', 'Granger Causal Links'],
    'Value': [f"{pca.explained_variance_ratio_[0]*100:.1f}%",
              f"{avg_lead:.0f} ± {1.96*se_lead:.0f} days",
              f"{d:.2f} ({'Large' if abs(d) > 0.8 else 'Medium'})",
              f"{len(error_df)}/{len(CRISES)}",
              f"{granger_df['significant'].sum()}/{len(granger_df)}"]
})
print(table1.to_string(index=False))

In [ ]:
# === FINAL EXPORT ===
import json

def make_serializable(obj):
    if hasattr(obj, 'isoformat'):
        return obj.isoformat()
    elif hasattr(obj, 'tolist'):
        return obj.tolist()
    elif isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, dict):
        return {str(k): make_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [make_serializable(i) for i in obj]
    else:
        return obj

publication_export = {
    'version': 'Great Caria Publication v1.0',
    'generated': pd.Timestamp.now().isoformat(),
    'methodology': {
        'factor_analysis': 'FactorAnalysis with PCA-informed components',
        'validation': ['Block Bootstrap', 'Granger Causality', 'PCMCI'],
        'signals': list(signals.columns)
    },
    'results': {
        'variance_explained': float(pca.explained_variance_ratio_[0]),
        'avg_lead_days': float(avg_lead),
        'lead_se': float(se_lead),
        'cohens_d': float(d),
        'crises_tested': len(error_df),
        'granger_significant': int(granger_df['significant'].sum()) if len(granger_df) > 0 else 0
    },
    'factor_loadings': make_serializable(loadings.to_dict('records')),
    'lead_times': make_serializable(error_df.to_dict('records')),
    'granger_results': make_serializable(granger_df.to_dict('records')) if len(granger_df) > 0 else [],
    'thresholds': {
        'warning': float(F_t.quantile(0.8)),
        'critical': float(F_t.quantile(0.95))
    }
}

with open(f'{OUTPUT_DIR}/publication_results.json', 'w') as f:
    json.dump(publication_export, f, indent=2)

print('\n✓ Exported: publication_results.json')

In [ ]:
# === FINAL SUMMARY ===
print('\n' + '='*70)
print('GREAT CARIA - PUBLICATION-READY MODEL')
print('='*70)

print('\n📊 DATA UNIVERSE:')
print(f'  Base signals: {len(["cf", "sync", "acf1", "var", "skew"])}')
print(f'  Expanded signals: {len(derived.columns)}')
print(f'  Total features: {signals.shape[1]}')

print('\n📈 MODEL PERFORMANCE:')
print(f'  Variance explained: {pca.explained_variance_ratio_[0]*100:.1f}%')
print(f'  Average lead time: {avg_lead:.0f} ± {1.96*se_lead:.0f} days')
print(f"  Effect size: Cohen's d = {d:.2f}")

print('\n🔬 STATISTICAL VALIDATION:')
print(f'  Block bootstrap: Complete')
print(f'  Granger causality: {granger_df["significant"].sum()}/{len(granger_df)} significant')
print(f'  Lead time CI: Computed for {len(error_df)} crises')

print('\n📁 SAVED FILES:')
print(f'  {OUTPUT_DIR}/')
print('    publication_figure.png')
print('    publication_figure.pdf')
print('    publication_results.json')

print('\n' + '='*70)
print('STATUS: Ready for publication and frontend integration')
print('='*70)